# Recommendations Engine For Movies

The moving rating data used in this excercise was taken from [Grouplens](https://grouplens.org/datasets/movielens/).

## Importing some Libraries

In [1]:
import pandas as pd
import numpy as np
import os
import re
from collections import Counter
from scipy.sparse.linalg import svds
from numpy.linalg import norm

## Extracting Titles from Reviews

In [17]:
class DataLoader:
    base_dir = "./cornell-data/scale_whole_review/scale_whole_review/{}/txt.parag"
    review_titles = {}
    
    def __init__(self, name):
        self.name = name
        if name == "Dennis+Schwartz":
            self.title_setter = self.title_setter_by_director_token
        else:
            self.title_setter = self.title_setter_by_frequence
    
    def filling_reviews_titles(self):
        directory = self.base_dir.format(self.name)
        #Selecting an title setter:
        review_files = os.listdir(directory)
        for review_file in review_files:
            review_text = self.open_file(directory,review_file)
            review_id = int(re.split("\.",review_file)[0])
            s = self.title_setter(review_text, review_id)
            
    def open_file(self,directory,review_file):
        with open("{}/{}".format(directory,review_file), "rb") as file:
            text_review = file.read().decode('utf-8',errors='ignore')
        return text_review
    
    def title_setter_by_director_token(self,review_text, review_id): 
        try:
            title = re.findall('([A-Z,.\:\-\!\/\'\s\d]+)\s+.*\([dD]irect', review_text)
            self.review_titles[review_id] = re.split("\,",title[0])[0]
        except:
            pass

    def title_setter_by_frequence(self,review_text, review_id):
        try:
            words_array = re.findall('([A-Z][A-Z\:\-\!\/\'\s\d\&]+) [a-z]',review_text)
            title = Counter(words_array).most_common(1)[0][0]
            self.review_titles[review_id] = title
        except:
            pass

In [18]:
names = ["Dennis+Schwartz","James+Berardinelli","Scott+Renshaw","Steve+Rhodes"]
for name in names:
    data_loader = DataLoader(name)
    data_loader.filling_reviews_titles()

## Creating a Movies DataFrame

In [19]:
movies_df = pd.DataFrame([DataLoader.review_titles],index=["movieId"]).T
movies_df.reset_index(inplace=True)
movies_df.rename(columns={"index":"reviewId"},inplace=True)
movies_df["title"] = movies_df["movieId"]

## Creating a Ratings DataFrame

In [21]:
id_base_dir = "./cornell-data/scale_data/scaledata/{}/id.{}"
rating_base_dir = "./cornell-data/scale_data/scaledata/{}/rating.{}"
ratings = pd.DataFrame([],columns=["reviewId","rating","userId"])
for name in names:
    rates = pd.read_table(rating_base_dir.format(name,name),names=["rating"])
    ids = pd.read_table(id_base_dir.format(name,name),names=["reviewId"])
    user_ratings = pd.concat([ids,rates],axis=1)
    user_ratings["userId"] = name
    ratings = pd.concat([ratings,user_ratings])
ratings_df = pd.merge(ratings,movies_df, on="reviewId")
ratings_df = ratings_df.groupby(["userId","movieId"]).head(1)

## Creating the User Item Table

To create the User - Item Matrix lets use a pivot table.

In [22]:
R_df = ratings_df.pivot(index="userId", columns="movieId", values="rating").fillna(-1)
R_df

movieId,UNMADE BEDS,13TH LETTER,15 MINUTES,200 CIGARETTES,2001: A SPACE ODYSSEY,3000 MILES TO GRACELAND,4D MAN,711 OCEAN DRIVE,8 1/2 WOMEN,8MM,...,YOU'VE GOT MAIL,YOUNG GIRLS OF ROCHEFORT,YOUR FRIENDS & NEIGHBORS,YOUR FRIENDS AND NEIGHBORS,Z,ZABRISKIE POINT,ZERO EFFECT,ZIP,ZOOLANDER,ZULU
userId,,,,,,,,,,,,,,,,,,,,,
Dennis+Schwartz,0.7,0.4,0.4,0.3,0.9,0.3,0.5,0.5,0.3,0.4,...,-1.0,0.8,-1.0,-1.0,0.5,0.8,0.6,-1.0,-1.0,0.8
James+Berardinelli,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.6,-1.0,-1.0,-1.0
Scott+Renshaw,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,0.7,-1.0,0.8,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
Steve+Rhodes,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,0.8,-1.0,-1.0,0.8,-1.0,-1.0,-1.0,0.7,0.8,-1.0


## Low Rank Matrix Factorization (Singular Value Decomposition)

In [23]:
R = R_df.as_matrix()
U,sigma,Vt = svds(R,k=np.min([(np.min(R.shape)-1)]))

### Selecting a User and a Number of recommendations

In [24]:
user = "Dennis+Schwartz"

### Selecting best ranked_movies

In [25]:
ratings = R_df.loc[user]
best_ranked_movies = ratings[ratings>0.7].sort_values(ascending=False).head(10)

### Selecting Features for unseen movies

In [26]:
Vt_df = pd.DataFrame(Vt,columns=R_df.columns)
unseen_movies = Vt_df[R_df.columns[ratings == -1]]

### Calculating Distances over Vt Matrix

In [27]:
differences = []
for movie in Vt_df[best_ranked_movies.index].items():
    differences.append(unseen_movies.apply(lambda x: norm(x-movie[1])).values)
distances = pd.DataFrame(differences,columns=unseen_movies.columns)

### Making some recommendations

In [28]:
recommendations_limit = 5
ds = list(set(distances.values.flatten()))
ds.sort()
indexes = distances.apply(lambda x: x.apply(lambda y: y in ds[:recommendations_limit]))
movie_lowest_score = distances[indexes].min()
movie_lowest_score[~movie_lowest_score.isnull()].sort_values().head(recommendations_limit)

movieId
FORCES OF NATURE         0.027885
THE HURRICANE            0.027885
THE END OF THE AFFAIR    0.027885
THE DINNER GAME          0.027885
IN & OUT                 0.027885
dtype: float64

## How to execute it

All this algorithm for a movie recommendation system is implemented in a python script. Below command executes the movies recommendation engine for the user with id 2, it making 6 recommendations, and showing the top 8 historical ratings.
```bash
python recommender.py '{"user id":"Steve+Rhodes", "Recommendations limit": 5, "Historical limit":8}'
```

## References

1. [Generals on movies recommendation systems.](https://blog.statsbot.co/recommendation-system-algorithms-ba67f39ac9a3)
1. [Matrix factorization recommender.](https://beckernick.github.io/matrix-factorization-recommender/)
2. [A movie recommendation system inplemented on Spark.](https://www.packtpub.com/books/content/building-recommendation-engine-spark)
3. [About the Netflix recommendation system.](https://medium.com/netflix-techblog/netflix-recommendations-beyond-the-5-stars-part-1-55838468f429)
4. [Performance metrics.](https://en.wikipedia.org/wiki/Information_retrieval#Precision_at_K)
5. [Movie ratings dataset.](https://grouplens.org/datasets/movielens/)